In [2]:
import pandas as pd
import numpy as np
from glob import glob
from pathlib import Path
from pydicom import dcmread
from skimage.io import imsave, imread
from skimage.transform import resize
import matplotlib.pyplot as plt
import random
import re
from IPython.display import clear_output
from functools import reduce

# Find some intersting imgs

In [6]:
paths = [Path(path) for path in glob('../breast/*/*/*/*/*/') if re.search(r't1', path, re.I)]
paths

[WindowsPath('../breast/manifest-1628804294858/Duke-Breast-Cancer-MRI/Breast_MRI_001/01-01-1990-NA-MRI BREAST BILATERAL WWO-97538/26.000000-ax t1 tse c-58582'),
 WindowsPath('../breast/manifest-1628804294858/Duke-Breast-Cancer-MRI/Breast_MRI_002/01-01-1990-NA-MRI BREAST BILATERAL W  WO-51972/3.000000-ax t1-40797'),
 WindowsPath('../breast/manifest-1628804294858/Duke-Breast-Cancer-MRI/Breast_MRI_003/01-01-1990-NA-MRI BREAST BILATERAL WWO-87435/3.000000-ax t1-86304'),
 WindowsPath('../breast/manifest-1628804294858/Duke-Breast-Cancer-MRI/Breast_MRI_004/01-01-1990-NA-MRI BREAST BILATERAL WWO-68914/3.000000-ax t1-53698'),
 WindowsPath('../breast/manifest-1628804294858/Duke-Breast-Cancer-MRI/Breast_MRI_005/01-01-1990-NA-BREASTROUTINE CA-77718/2.000000-ax t1-53433'),
 WindowsPath('../breast/manifest-1628804294858/Duke-Breast-Cancer-MRI/Breast_MRI_006/01-01-1990-NA-MRI BREAST BILATERAL WWO-27236/3.000000-ax t1-43781'),
 WindowsPath('../breast/manifest-1628804294858/Duke-Breast-Cancer-MRI/Breas

In [7]:
paths = reduce(lambda a, b: a+b,  [glob(str(path / '*')) for path in paths])

In [8]:
ds = dcmread(paths[0])
ds

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 204
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.6.1.4.1.14519.5.2.1.210062012468375413079275155624796278039
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.3.6.1.4.1.22213.1.143
(0002, 0013) Implementation Version Name         SH: '0.5'
(0002, 0016) Source Application Entity Title     AE: 'POSDA'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M', 'ND']
(0008, 0012) Instance Creation Date              DA: '19900101'
(0008, 0013) Instance Creation Time              TM: '084915.734000'
(0008, 0016) SOP Class UID  

In [9]:
names = [str(dcmread(path).PatientName) for path in paths]
pd.Series(names).value_counts()

Breast_MRI_007    224
Breast_MRI_004     90
Breast_MRI_003     64
Breast_MRI_002     60
Breast_MRI_009     58
Breast_MRI_005     58
Breast_MRI_006     53
Breast_MRI_010     52
Breast_MRI_001     42
Breast_MRI_008     37
dtype: int64

In [11]:
sizes = []
for i, path in enumerate(paths):
    y = dcmread(path).pixel_array
    sizes.append(y.shape)
    if(y.min() < y.max()):  
        y = (y - y.min())/(y.max() - y.min()) 
    z = y - y.mean()  
    imsave(
        f'not_resized/breast/{str(i).zfill(6)}.jpeg', 
        (z.clip(0, 1) * 255).astype(np.uint8))
    clear_output(wait=True)
    print(f'{i+1}/{len(paths)} ({(i+1)/len(paths)*100:.2f})')        

738/738 (100.00)


In [12]:
pd.Series(sizes).value_counts()

(512, 512)    287
(448, 448)    224
(256, 256)    169
(320, 320)     58
dtype: int64